# PROYECTO FINAL GRUPAL  - NBA-StatVision - Lesiones


### Importacion de librerias


In [5]:
import pandas as pd 
import numpy as np 


Importamos dataframe "play_by_play" pero solo leemos la primer fila que es el nombre de la columna


In [6]:
## creamos variable para alojar la ruta del archivo
ruta_archivo_csv = r'C:\Users\Fernando\Desktop\NBA Statvision\archive\csv\play_by_play.csv'

# df = pd.read_csv(ruta_archivo_csv)

## Leemos solo la primer fila con el nombre de las columnas.
def obtener_nombre_columnas(ruta_archivo_csv):
    
    columnas = pd.read_csv(ruta_archivo_csv, nrows=0).columns.tolist()
    return columnas




Vemos los nombres de las columnas de "play_by_play" (esto es porque nos trae la primer fila del dataset)

In [7]:
primera_fila = obtener_nombre_columnas(ruta_archivo_csv)
# mostramos las columnas:
print("Columnas del archivo 'play_by_play': ")
for col in primera_fila:
    print("-", col)

Columnas del archivo 'play_by_play': 
- game_id
- eventnum
- eventmsgtype
- eventmsgactiontype
- period
- wctimestring
- pctimestring
- homedescription
- neutraldescription
- visitordescription
- score
- scoremargin
- person1type
- player1_id
- player1_name
- player1_team_id
- player1_team_city
- player1_team_nickname
- player1_team_abbreviation
- person2type
- player2_id
- player2_name
- player2_team_id
- player2_team_city
- player2_team_nickname
- player2_team_abbreviation
- person3type
- player3_id
- player3_name
- player3_team_id
- player3_team_city
- player3_team_nickname
- player3_team_abbreviation
- video_available_flag


Este data set no tiene datos de fecha, solo 'game_id', entonces deberemos buscar en otro dataset, si hay game_id con fechas.

Buscamos en el dataset 'game_info' para ver si contiene una columna de fechas.

In [8]:
## Definimos la ruta donde se encuentra
ruta_game_info = r'C:\Users\Fernando\Desktop\NBA Statvision\archive\csv\game_info.csv' 

## Leemos el dataset
df = pd.read_csv(ruta_game_info)

## Lo mostramos 
df.head(2000)


,game_id,game_date,attendance,game_time
0,24600001,1946-11-01 00:00:00,NaN,NaN
1,24600003,1946-11-02 00:00:00,NaN,NaN
2,24600002,1946-11-02 00:00:00,NaN,NaN
3,24600004,1946-11-02 00:00:00,NaN,NaN
4,24600005,1946-11-02 00:00:00,NaN,NaN
...,...,...,...,...
1995,25200088,1952-12-05 00:00:00,NaN,NaN
1996,25200090,1952-12-06 00:00:00,NaN,NaN
1997,25200092,1952-12-06 00:00:00,NaN,NaN
1998,25200091,1952-12-06 00:00:00,NaN,NaN


Como tiene columna de fechas, vamos a filtrar los id que estén entre 2013 y 2023

In [12]:
# Cargamos nuevamente el data set, para "parsear" la columna 'game_date'.
df_games = pd.read_csv(ruta_game_info, parse_dates=['game_date'])

# Filtrar por fechas:

games_filtred = df_games [
    (df_games['game_date'] >= '2013-01-01') &
    (df_games['game_date'] <= '2023-01-01')
]

# obtener el set de 'game_id' válidos
game_ids_validos = set(games_filtred['game_id'])
print(f'Cantidad de partidos entre 2013 y 2023: {len(game_ids_validos)}')

Cantidad de partidos entre 2013 y 2023: 11714


 Uso de Chunksize para filtrar el dataset de 'play_by_play' con los 'game_id' obtenidos

In [15]:
# Creamos una lista para los Chunks

chunks = []

for chunk in pd.read_csv(r'C:\Users\Fernando\Desktop\NBA Statvision\archive\csv\play_by_play.csv', chunksize=100000):
    chunk_filtrado = chunk[chunk['game_id'].isin(game_ids_validos)]
    chunks.append(chunk_filtrado) ## llena la lista 

# Concatenamos los chunks
play_by_play_filtrado = pd.concat(chunks, ignore_index=True)

# mostramos el dataset filtrado
print(play_by_play_filtrado.shape)
play_by_play_filtrado.head()





(4940642, 34)


,game_id,eventnum,eventmsgtype,eventmsgactiontype,period,wctimestring,pctimestring,homedescription,neutraldescription,visitordescription,...,player2_team_nickname,player2_team_abbreviation,person3type,player3_id,player3_name,player3_team_id,player3_team_city,player3_team_nickname,player3_team_abbreviation,video_available_flag
0,41200161,0,12,0,1,5:41 PM,12:00,NaN,Start of 1st Period (5:41 PM EST),NaN,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
1,41200161,1,10,0,1,5:41 PM,12:00,Jump Ball Koufos vs. Bogut: Tip to Lawson,NaN,NaN,...,Warriors,GSW,4.0,201951,Ty Lawson,1.610613e+09,Denver,Nuggets,DEN,0
2,41200161,2,2,42,1,5:42 PM,11:40,MISS Fournier Driving Layup,NaN,Bogut BLOCK (1 BLK),...,NaN,NaN,5.0,101106,Andrew Bogut,1.610613e+09,Golden State,Warriors,GSW,0
3,41200161,3,4,0,1,5:42 PM,11:39,NaN,NaN,Bogut REBOUND (Off:0 Def:1),...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
4,41200161,4,2,1,1,5:42 PM,11:27,NaN,NaN,MISS Lee 15' Jump Shot,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0


Guardamos el dataset en un archivo nuevo de .CSV 


In [16]:
play_by_play_filtrado.to_csv('juego_por_juevo.csv', index=False)
#Se guarda el archivo nuevo ya con los id de juego pertenecientes al rango de fecha que especificamos.
